<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [81]:
import pandas as pd
import psycopg2

In [82]:
import settings
import local_settings
connection = psycopg2.connect(
    dbname=local_settings.DBNAME_PROJECT_SQL,
    user=local_settings.USER,
    host=local_settings.HOST,
    password=local_settings.PASSWORD,
    port=local_settings.PORT
)

In [83]:
def run_query(text_query):
    df = pd.read_sql(text_query, connection)
    display(df)
    
import warnings
warnings.filterwarnings('ignore')

# Юнит 3. Предварительный анализ данных

1. Посчитаем количество вакансий в нашей базе

In [9]:
run_query(f'''
    SELECT count(*) AS vacancies_count
    FROM vacancies
''')


,vacancies_count
0,49197


2. Посчитаем количество работодателей 

In [10]:
run_query(f'''
    SELECT count(*) AS employers_count
    FROM employers
''')

,employers_count
0,23501


3. Посчитаем количество регионов

In [11]:
run_query(f'''
    SELECT count(*) AS areas_count
    FROM areas
''')

,areas_count
0,1362


4. Посчитаем количество сфер деятельности

In [16]:
run_query(f'''
    SELECT count(*) AS industries_count
    FROM industries
''')

,industries_count
0,294


***

В выборке представлен широкий список вакансий (49197). Это и различные сферы деятельности (294), и различные регионы (1362), и большое количество работодателей (23501).

Некоторые работодатели открыли несколько вакансий.

# Юнит 4. Детальный анализ вакансий

1. Узнаем, сколько вакансий в каждом регионе

In [21]:
# выведем для анализа 10 лидеров вместо 5
run_query(f'''
    SELECT a.name AS area, count(*) AS cnt
    FROM vacancies v
    LEFT JOIN areas a ON a.id = v.area_id
    GROUP BY a.name
    ORDER BY cnt DESC
    LIMIT 10
''')

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Определим, у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой

In [22]:
run_query(f'''
    SELECT count(*) AS vacancies_count
    FROM vacancies
    WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
''')

,vacancies_count
0,24073


3. Найдем средние значения для нижней и верхней границы зарплатной вилки

In [23]:
run_query(f'''
    SELECT 
        ROUND(AVG(salary_from), 2) as avg_salary_from,
        ROUND(AVG(salary_to), 2) AS avg_salary_to
    FROM vacancies
''')

,avg_salary_from,avg_salary_to
0,71064.66,110536.74


4. Выведем количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства, используемого в вакансиях


In [26]:
run_query(f'''
    SELECT schedule, employment, count(*) AS cnt
    FROM vacancies 
    GROUP BY schedule, employment
    ORDER BY cnt DESC
''')

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Узнаем количество вакансий в зависимости от требуемого опыта

In [27]:
run_query(f'''
    SELECT experience, count(*) AS cnt
    FROM vacancies 
    GROUP BY experience
    ORDER BY cnt
''')

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

1. Представлены не только вакансии в России, но и в других странах. Самое большое количество вакансий расположено в столицах и крупных городах. Лидируют Москва и Санкт-Петербург. На долю первых 6-ти крупных городов (Москва, Санкт-Петербург, Минск, Новосибирск, Алматы, Екатеринбург) приходится треть всех вакансий;

2. У половины вакансий не заполнена заработная плата. Вероятно, заработная плата работодателем назначается либо по результатам собеседования, либо озвучивается только лично кандидату;

3. В среднем работодатели указывают зарплатную вилку от 71 до 110 тысяч. Если это суммы до вычета НДФЛ, то на руки работники получат в среднем от 62 до 96. Скорее всего, в регионах суммы гораздо меньше, на среднее в большей степени влияют суммы вакансий в крупных городах (их подавляющее большинство), а также вакансии в других странах;

4. В большинстве вакансий (72%) требуются сотрудники на полный день с полной занятостью. В 16% вакансий работодатели готовы принять сотрудников на удаленную работу на полный день, в 3% на гибкий график. Остальные комбинации графика раоты и типа трудоустройства гораздо менее популярны. Это закономерно, работодатели считают очную работу более производительной, нежели удаленную. И большая часть предприятий работают по стандартному графику 8 часов в день, 5 дней в неделю;

5. Больше половины вакансий рассчитаны на кандидатов с небольшим опытом (1-3 года). В 29% хотят видеть более опытных кандидатов (3-6 лет). Кандидатов без опыта готовы рассмотреть только в 15% вакансий, т.к. на подготовку таких работников требуется слишком много ресурсов компании. Слишком большой требуемый опыт работодатели, вероятнее всего, не указывают, чтобы не отпугнуть кандидатов и не уменьшать шансы найти нужного.

# Юнит 5. Анализ работодателей

1. Узнаем, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [67]:
# Можно ограничить выводимые записи с помощью LIMIT и OFFSET, как указанно в закомментированных строках запроса.
# Вывела 10 лидеров для анализа 
run_query(f'''
    SELECT e.name AS employer_name, count(v.id) as vacancies_count
    FROM employers e
    LEFT JOIN vacancies v on e.id = v.employer_id
    GROUP BY e.name
    ORDER BY vacancies_count DESC
    LIMIT 10
    --LIMIT 1
    --LIMIT 1 OFFSET 4
''')   

,employer_name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


2. Для каждого региона выведем количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдем тот, в котором наибольшее количество работодателей


In [59]:
# для анализа выведем лидеров по количеству работодателей
run_query(f'''
    SELECT 
        a.name AS area_name, 
        sum(COALESCE(employers_count, 0)) AS employers_count, 
        sum(COALESCE(vacancies_count, 0)) AS vacancies_count
    FROM areas a
    LEFT JOIN (
        SELECT area AS area_id, count(distinct id) AS employers_count, 0 as vacancies_count
        FROM employers
        GROUP BY area
    
        UNION all
    
        SELECT area_id, 0, count(distinct id)
        FROM vacancies
        GROUP BY area_id) ev ON a.id = ev.area_id
    GROUP BY a.name
    ORDER BY employers_count DESC, vacancies_count DESC
    LIMIT 10
''')   

,area_name,employers_count,vacancies_count
0,Москва,5864.0,5333.0
1,Санкт-Петербург,2217.0,2851.0
2,Минск,1115.0,2112.0
3,Алматы,721.0,1892.0
4,Екатеринбург,609.0,1698.0
5,Новосибирск,573.0,2006.0
6,Казань,480.0,1415.0
7,Нижний Новгород,426.0,1670.0
8,Россия,410.0,0.0
9,Краснодар,409.0,1301.0


In [62]:
# Среди регионов, в которых нет вакансий, найдем тот, в котором наибольшее количество работодателей
# Можно отсортировать результат предыдущего запроса или сформировать запрос по-другому
run_query(f'''
    SELECT 
        a.name AS area_name, 
        count(e.id) AS employers_count
    FROM areas a
    INNER JOIN (
        SELECT id AS area_id 
        FROM areas
    
        EXCEPT
    
        SELECT area_id
        FROM vacancies) ua on ua.area_id = a.id
    LEFT JOIN employers e ON a.id = e.area
    GROUP BY a.name
    ORDER BY employers_count DESC
    LIMIT 10
''')   

,area_name,employers_count
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Ростовская область,18
5,Беларусь,18
6,Азербайджан,17
7,Нижегородская область,16
8,Республика Татарстан,16
9,Узбекистан,15


3. Для каждого работодателя посчитаем количество регионов, в которых он публикует свои вакансии

In [54]:
run_query(f'''
    SELECT e.name AS employer_name, count(distinct v.area_id) as areas_count
    FROM employers e
    LEFT JOIN vacancies v on e.id = v.employer_id
    GROUP BY e.name
    ORDER BY areas_count DESC
    LIMIT 10
''')   

,employer_name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Посчитаем количества работодателей, у которых не указана сфера деятельности

In [53]:
# Можно также левым соединением и условием на NULL
run_query(f'''
    SELECT count(*) 
    FROM (
        SELECT id
        FROM employers
        
        EXCEPT
        
        SELECT employer_id
        FROM employers_industries
    ) emp
''')   

,count
0,8419


5. Узнаем название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [143]:
# Вывела первые 10
run_query(f'''
    SELECT e.name as employer_name, count(ei.industry_id) as industry_count
    FROM employers_industries ei
    INNER JOIN employers e on ei.employer_id = e.id
    GROUP BY e.name
    HAVING count(ei.industry_id) = 4
    ORDER BY e.name
    LIMIT 1 OFFSET 2
''')   

,employer_name,industry_count
0,2ГИС,4


In [146]:
# Выведем первые 10 лидеров по количеству сфер деятельности
run_query(f'''
    SELECT e.name as employer_name, count(ei.industry_id) as industry_count
    FROM employers_industries ei
    INNER JOIN employers e on ei.employer_id = e.id
    GROUP BY e.name
    ORDER BY count(ei.industry_id) DESC
    LIMIT 4
''')   

,employer_name,industry_count
0,Модуль,16
1,Простые решения,11
2,Энергия,10
3,Прайд,10


6. Выясним, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [52]:
run_query(f'''
    SELECT count(distinct ei.employer_id)
    FROM employers_industries ei
    INNER JOIN industries i on ei.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
''')   

,count
0,3553


7. Для компании «Яндекс» выведем список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах и общими итогами

In [88]:
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
paragraphs = page.find('table', class_='sortable').find('tbody').find_all('td')
cities = [paragraph.find('a')['title'] for paragraph in paragraphs if paragraph.find('a')]
cities_tuple = tuple(cities)
run_query(f'''
    WITH x AS (SELECT id as emp_id FROM employers WHERE name = 'Яндекс'),
    y AS (SELECT a.name, count(v.id) as vacancies_count
        FROM vacancies v
        INNER JOIN areas a ON v.area_id = a.id
            AND v.employer_id = (SELECT emp_id FROM x) 
            AND a.name in {cities_tuple}
        GROUP BY a.name)
    (SELECT * FROM y ORDER BY vacancies_count)
    UNION ALL
    SELECT 'Total', sum(vacancies_count) FROM y
''')

,name,vacancies_count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

1. Несомненным лидером по количеств вакансий является Яндекс (1933 вакансии, 8% всех вакансий). Далее следуют такие крупные компании, как Ростелеком, Тинькофф, СБЕР, Газпром нефть.

2. Наибольшее количество работодателей в Москве (25%). Далее следует Санкт-Петербург, Минск и Алматы. Примерно таких же лидеров мы видели и в топе регионов по количеству вакансий. Есть регионы, в которых представлены работодатели, но нет вакансий. В основном такое наблюдается в регионах "Москва" и "Казахстан". Вероятно, работодатели, которые имеют филиалы или размещают удаленные вакансии в нескольких регионах, указывают в профайле общий регион, а в вакансиях уже конкретный;

3. Вакансии в наибольшем количестве регионов представлены у таких крупных компаний, как Яндекс и Ростелеком. Также неожиданно в лидерах по количеству регионов оказались компания ООО "СпецремонтЭ и ИП Поляков Денис Иванович, которые занимаются ремонтом компьютеров и ищут соответствующих специалистов;

4. У 8419 работодателей (36%) не указана сфера деятельности. Это может исказить результаты анализа сфер деятельности работодателей;

5. У некоторых компаний указано несколько сфер деятельности. Максимум - 16;

6. Разработкой программного обеспечения занимаются 3553 работодателя (15% среди всех работодателей и 24% из указавших сферу деятельности). Развитие в этом направлении перспективно с точки зрения трудоустройства;

7. Компания Яндекс представила вакансии в 181 регионе, как мы видели ранее. 16 из этих регионов - города-миллионники. Стоит помнить, что мы учли только российские миллионники, а ввакансиях имются и города других стран. Максимальное число вакансий среди городов-миллионников представлено в Москве, что вполне закономерно из-за размера города.


# Юнит 6. Предметный анализ

1. Посчитаем, сколько вакансий имеет отношение к данным (в её названии содержатся слова 'data' или 'данн')


In [28]:
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE LOWER(name) LIKE '%данн%' OR LOWER(name) LIKE '%data%'
''')    

,cnt
0,1771


2. Посчитаем, сколько есть подходящих вакансий для начинающего дата-сайентиста

In [102]:
# вакансии для начинающего специалиста
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE (LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR name LIKE '%ML%' AND name NOT LIKE '%HTML%')
        AND (LOWER(name) SIMILAR TO '%junior%' OR experience = 'Нет опыта' OR employment = 'Стажировка')
''')    

,cnt
0,51


In [103]:
# вакансии по DS всего
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE (LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR name LIKE '%ML%' AND name NOT LIKE '%HTML%')
''') 

,cnt
0,480


3. Узнаем, сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres

In [90]:
run_query(f'''
    SELECT count(*)
    FROM vacancies 
    WHERE (LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR name LIKE '%ML%' AND name NOT LIKE '%HTML%')
        AND LOWER(key_skills) SIMILAR TO '%(sql|postgres)%'
''')  

,count
0,201


4. Проверим, насколько популярен Python в требованиях работодателей к DS

In [91]:
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE (LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR name LIKE '%ML%' AND name NOT LIKE '%HTML%')
        AND LOWER(key_skills) SIMILAR TO '%python%'
''')  

,cnt
0,351


5. Проверим, сколько ключевых навыков в среднем указывают в вакансиях для DS

In [96]:
run_query(f'''
    SELECT ROUND(AVG((CHAR_LENGTH(key_skills)-CHAR_LENGTH(REPLACE(key_skills, E'\t',''))+1)), 2) AS cnt
    FROM vacancies 
    WHERE (LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR name LIKE '%ML%' AND name NOT LIKE '%HTML%')
''')   

,cnt
0,6.41


6. Вычислим, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*)

In [100]:
run_query(f'''
    SELECT experience, 
        ROUND(AVG(
            (COALESCE(salary_from, 0) + COALESCE(salary_to, 0))/
            CASE WHEN salary_from IS NULL OR salary_to IS NULL THEN 1 ELSE 2 END
        ), 2) AS avg_salary
    FROM vacancies 
    WHERE (LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR name LIKE '%ML%' AND name NOT LIKE '%HTML%')
        AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    GROUP BY experience
    ORDER BY avg_salary DESC
''')      

,experience,avg_salary
0,От 3 до 6 лет,243114.67
1,От 1 года до 3 лет,139674.75
2,Нет опыта,74642.86


***

1. Найденно 1771 вакансий, предположительно связанных с данными. Сюда входят не только специалисты DS в чистом виде, но администраторы баз данных, аналитики всех видов. DS - это быстроразвивающееся направление, но пока вакансии по работе с данными составляют менее 4% всех вакансий;

2. У начинающих дата-сайентистов не так много возможностей. Найдено всего 51 таких вакансий из 480 вакансий по DS (11%). Видимо, не все работодатели готовы и имеют ресурсы на обучение начинающих специалистов;

3. SQL указан в качестве ключевого навыка в 201 вакансиях DS из 480 (42%). Соответственно специалисту DS обязательно нужно иметь хорошие навыки работы с запросами. А PostgreSQL - одна из самых распространенных баз данных, т.к. является бесплатной;

4. Знания Python требуются в 351 вакансиях DS из 480 (73%). Т.о. без знаний Python практически невозможно устроиться в DS;

5. В среднем работодатели указывают в вакансиях 6-7 ключевых навыков. Т.о. специалист по DS должен быть достаточно разностороннеразвитым. Это может быть одной из причин небольшого числа вакансий для начинающего специалиста;

6. Средняя заработная плата коррелирует с опытом кандидатов. Самая высокая средняя зарплата в DS у кандидатов с опытом более 3-х лет. Заработная плата кандидатов даже с опытом от 1 года (140 тыс) выше, чем верхняя планка средней заработной платы по всем вакансиям, которую мы рассчитывали ранее. Вероятно, это связано с тем, что эта сфера требует много профессиональных навыков и знаний.

# Общий вывод по проекту

1. Проанализируем средние заработные платы по регионам

In [110]:
run_query(f'''   
    SELECT a.name, s.avg_salary, s.cnt FROM
        (SELECT area_id, count(*) as cnt,
            ROUND(AVG(
                (COALESCE(salary_from, 0) + COALESCE(salary_to, 0))/
                CASE WHEN salary_from IS NULL OR salary_to IS NULL THEN 1 ELSE 2 END
            ), 2) AS avg_salary
        FROM vacancies 
        WHERE (salary_from IS NOT NULL OR salary_to IS NOT NULL)
        GROUP BY area_id) s
    INNER JOIN areas a ON a.id = s.area_id
    ORDER BY avg_salary DESC
    LIMIT 15
''')

,name,avg_salary,cnt
0,Германия,452751.00,2
1,Болгария,324348.00,3
2,Испания,300000.00,1
3,Черногория,288589.25,4
4,Литва,263018.50,2
5,Польша,257155.67,3
6,Петропавловка (Республика Бурятия),250000.00,1
7,Турция,235435.42,19
8,Кипр,223735.59,39
9,Кутаиси,220474.67,3


Как мы видим, в топе вакансии в других странах. В некоторых странах их совсем немного, но эти данные могут исказить статистику по средним заработным платам в России. Даже Москва (134 тыс) в этом рейтинге только на 40-м месте. 

При анализе в предыдущих блоках мы искали среднюю заработную плату по всем регионам, а заработную плату DS в городах-миллионниках только с учетом городов России. Соответственно эти данные некорректно сравнивать между собой.

2. Проведем анализ средних заработных плат в других областях IT

In [133]:
query = f'''
    SELECT experience, 
        ROUND(AVG(
            (COALESCE(salary_from, 0) + COALESCE(salary_to, 0))/
            CASE WHEN salary_from IS NULL OR salary_to IS NULL THEN 1 ELSE 2 END
        ), 2) AS avg_salary,
        count(*) AS cnt
    FROM vacancies 
    WHERE (salary_from IS NOT NULL OR salary_to IS NOT NULL) AND &industry_condition
    GROUP BY experience
    ORDER BY avg_salary DESC
'''

In [134]:
# Сначала посмотрим на ЗП по всем вакансиям, где она указана
run_query(query.replace('&industry_condition', 'True'))

,experience,avg_salary,cnt
0,Более 6 лет,166256.08,550
1,От 3 до 6 лет,133210.92,5649
2,От 1 года до 3 лет,76540.82,12819
3,Нет опыта,40471.95,5055


In [136]:
# Теперь на ЗП Python-разработчиков
run_query(query.replace('&industry_condition', "LOWER(name) LIKE '%python%'"))

,experience,avg_salary,cnt
0,Более 6 лет,188712.80,10
1,От 3 до 6 лет,185162.61,101
2,От 1 года до 3 лет,114182.08,144
3,Нет опыта,78186.83,18


In [137]:
# Теперь на ЗП Python-разработчиков
run_query(query.replace('&industry_condition', "LOWER(name) LIKE '%1с%'"))

,experience,avg_salary,cnt
0,Более 6 лет,162024.02,60
1,От 3 до 6 лет,133794.00,707
2,От 1 года до 3 лет,94283.59,1250
3,Нет опыта,49750.24,262


Мы видим, общие средние заработные платы получились довольно высокими. Это, вероятно, связано с тем, что среди вакансий есть вакансии в других странах. 

Также видим, что во всех направлениях специалистов с опытом более 6-ти лет ищут довольно редко. Вероятно, работодатели не хотят отпугивать такими ограничениями потенциальных работников.

Заработные платы Python-разработчиков выше, чем средние заработные платы по всем вакансиям. Однако вакансий c упоминание Python в названии не так много (в качестве навыка Python указан в гораздо большем числе вакансий, но они слишком разноплановые). Удивительно, что зарплата специалистов с опытом в Python более 6 лет практически идентична заработной плате специалистов от 3-х лет. 

Также удивительно, что средняя заработная плата специалистов по 1С сопоставима со средней в целом. В России это, скорее всего, не так. На такую статистику влияет то, что в других странах 1С практически не используется.

Т.о. заработные платы в DS самые высокие.

3. Посмотрим, требуется ли знание английского при работе в DS

In [126]:
run_query(f'''  
   SELECT a.name, count(*)
    FROM vacancies v
    LEFT JOIN areas a ON v.area_id = a.id
    WHERE 
        LOWER(key_skills) SIMILAR TO '%(английск|english)%'
        AND (LOWER(v.name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    GROUP BY a.name
''')

,name,count
0,Алматы,1
1,Армения,3
2,Батуми,1
3,Кипр,2
4,Минск,3
5,Москва,20
6,Новосибирск,2
7,Нур-Султан,2
8,Пенза,1
9,Ростов-на-Дону,1


Английский требуется в 50 вакансиях DS из 480 (10%). Примечательно, что это не только зарубежные вакансии. Больше половины вакансий, в которых работодатели указали английский, находятся в России. Т.о., зная английский, кандидаты могут немного повысить свои шансы на трудоустройство.

4. Приблизительно прикинем количество IT-вакансий в выборке

In [138]:
run_query(f'''    
    SELECT count(*)
    FROM vacancies v
    WHERE  (LOWER(name) SIMILAR TO '%(разработ|программист|teamlead|аналитик|тестиров|developer|devops|analyst|специалист ит|системный администрато)%'
    OR (LOWER(v.name) SIMILAR TO '%(data scientist|data science|исследователь данных|machine learning|машинн%обучен)%' 
            OR v.name SIMILAR TO '%(ML|QA)%' AND v.name NOT LIKE '%HTML%'))
''')

,count
0,28062


Представленный запрос только приблизительно показывает объем вакансий IT. Есть много специфичных названий. 

Либо предоставленные данные - это выборка именно IT-специальностей, либо IT-специалисты - это наиболее популярная специальность.
_________________________________________________________________________________________

Подведем итоги.

Наибольшее число вакансий представлено в крупных городах. Только в Москве представлено 11% всех вакансий, а в Санкт-Петербурге еще 6%. Имеются вакансии в разных странах.

Заработные платы за рубежом в среднем выше, чем в России. А в крупных городах России выше, чем в других регионах.

Есть крупные работодатели, которые ищут специалистов во многих городах. Либо такие компании имеют филилалы по всей стране, либо готовы на удаленную работу. Например, Яндекс (152 регион), Ростелеком (181 регион). Эти компании также являются лидерами по числу вакансий 1933 и 491 соответственно (5% от всех вакансий).

Идеальный кандидат для работодателя - это работник, готовый к полной занятости и имеющий опыт от 1 до 6 лет.

IT - это быстро развивающаяся и наиболее перспективная сфера деятельности. Подавляющее большинство вакансий так или иначе связаны именно с IT.

Среди IT специалистов все больше начинают требоваться именно специалисты DS. Заработные платы таких специалистов одни из самых высоких. Однако есть сложности со стартом в этом направлении, вакансий для начинающих специалистов не так много.

Начинающему специалисту DS необходимо знать, как минимум, SQL и Python. Однако этого не достаточно для трудоустройства. В среднем работодатели указывают 6-7 требуемых навыком. Скорее всего, это могут быть знания математики, ML, а также знания необходимых технологий и программ.

Стоит отметить, что при анализе заработных плат мы учитывали также вакансии других стран, а также исключали из анализа те вакансии, где заработная плата не была указана. В вакансиях руководителей и прочих высоких должностей редко указываются заработные платы. Поэтому рассчитанное нами среднее не в полной мере отражает реальную ситуацию.

Для продолжения исследования можно в таблицу Areas добавить и заполнить колонку Страна. Это позволило бы анализировать данные в разрезе стран, а также исключать другие страны при анализе заработных плат в России. Также представляет сложность анализ по видам деятельности компаний, т.к. уникальных видов слишком много, у каждой компании может быть несколько видов деятельности (до 16), а также у некоторых компаний вид деятельности вообще не указан.